In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from magpylib import magnet
from nn_magnetics.utils.physics import (
    Nz_elementwise,
    Nz,
    magnet_cuboid_Bfield_homogeneous,
)
from nn_magnetics.utils.metrics import angle_error, relative_amplitude_error, calculate_metrics_baseline
from nn_magnetics.data.create_data import generate_points_grid
import numpy as np
from nn_magnetics.data import AnisotropicData
from nn_magnetics.models import QuaternionNet, AngleAmpCorrectionNetwork
from pathlib import Path
import torch
from nn_magnetics.utils.plotting import plot_heatmaps
from time import perf_counter
from nn_magnetics.predictions import B as calcB
import magpylib
from magpylib_material_response import meshing, demag

In [ ]:
torch.set_printoptions(precision=8)

In [36]:
valid_data = AnisotropicData("../data/3dof_chi_v2/validation_fast", device="cpu")
X, B = valid_data.get_magnets()
X, B = X[:1, 5, :], B[:1, 5, :]

In [37]:
angle_err, amp_err = calculate_metrics_baseline(B)
torch.mean(angle_err), torch.mean(amp_err)

(tensor(3.21726561e-05), tensor(6.75707207))

In [38]:
for Xi, Bi in zip(X, B):
    a, b = Xi[0], Xi[1]
    chix, chiy, chiz = Xi[2], Xi[3], Xi[4]
    x, y, z = Xi[5], Xi[6], Xi[7]

    _, B2 = Bi[:3], Bi[3:]
    print(Bi)

    Dz = Nz_elementwise(a=a.item(), b=b.item(), c=1)

    # my solution
    B1 = magnet_cuboid_Bfield_homogeneous(
        observers=torch.tensor([[x, y, z]]),
        dimensions=torch.tensor([[a, b, 1]]),
        polarizations=torch.tensor([[0, 0,1 / (1 + chiz * Dz) ]]),
    )[0].numpy()

    cuboid = magnet.Cuboid(dimension=(a, b, 1), polarization=(0, 0, 1))

    mesh = meshing.mesh_Cuboid(cuboid=cuboid, target_elems=500)
    demag.apply_demag(mesh, (chix, chiy, chiz), inplace=True)
    M = mesh.getM([m.position for m in mesh])
    J = magpylib.mu_0 * np.mean(M, axis=0)
    cuboid2 = magnet.Cuboid(dimension=(a, b, 1), polarization=J)
    B4 = cuboid2.getB([x, y, z])

    reduced_polarization = (0, 0, 1 / (1 + chiz * Dz))
    magnet_reduced = magnet.Cuboid(
        polarization=reduced_polarization,
        dimension=(a, b, 1),
    )
    B5 = magnet_reduced.getB([x, y, z])

    # print(f"a={a}, b={b}, c=1.0, chi={chix.item(), chiy.item(), chiz.item()}, obs={x.item(), y.item(), z.item()}, dmf={dmf}")
    print(f"Torch B:\tB={B1}") # my implementation of getB in torch w/ reduced P
    print(f"Precomputed B:\tB={B2}") # precomputed B_reduced
    print(f"Magpylib with calculated J={J[2]} from M:\tB={B4}") # getB from reduced P demag
    print(f"Magpylib with new J={1 / (1 + chiz * Dz)}:\tB={B5}")
    print("=" * 50)

tensor([1.01448185e-06, 5.79067633e-07, 1.15859365e-01, 1.14048974e-06,
        6.57149281e-07, 1.23688066e-01])
2025-03-26 at 13:09:13 |   INFO   | run | {} ℹ️ Start Demagnetization (inplace) of Collection(id=5210314736) with 495 cells - Counter({'Cuboid': 495})
2025-03-26 at 13:09:14 | SUCCESS  | timelog | {} ✅  Demagnetization (inplace) of Collection(id=5210314736) with 495 cells - Counter({'Cuboid': 495}) done 🕑 1.069sec
Torch B:	B=[7.48300386e-07 3.09876684e-07 1.23688066e-01]
Precomputed B:	B=tensor([1.14048974e-06, 6.57149281e-07, 1.23688066e-01])
Magpylib with calculated J=0.7633934811547399 from M:	B=[7.54358433e-07 3.12385366e-07 1.24689412e-01]
Magpylib with new J=0.7572628766091738:	B=[7.48300386e-07 3.09876684e-07 1.23688066e-01]


In [ ]:
a = 0.5796
b = 1.9572
chi = (0.0695, 0.1402, 0.9083)

cuboid = magpylib.magnet.Cuboid(dimension=(a, b, 1), polarization=(0, 0, 1))

mesh = meshing.mesh_Cuboid(cuboid=cuboid, target_elems=100)
demag.apply_demag(mesh, susceptibility=chi, inplace=True)

M = mesh.getM([m.position for m in mesh])
J = magpylib.mu_0 * np.mean(M, axis=0)

cuboid2 = magpylib.magnet.Cuboid(dimension=(a, b, 1), polarization=J)
obs = [1.6000, 2.4000, 2.4000]

B2 = cuboid2.getB(obs)
B2